# DZ locations normalizer

Thanks to Alexey Galda (GitHub: @alexgalda) and wingsuitworld.com for providing the raw CSV of drop zone locations and elevations where speeders and wingsuit pilots train and compete.


## FCC sexagecimal to decimal degrees converter:

https://www.fcc.gov/media/radio/dms-decimal

In [ ]:
import pandas as pd

## Get the raw data

In [ ]:
RAW_DATA_FILE = './resources/drop-zones-loc-elev-raw.csv'
rawData = pd.read_csv(RAW_DATA_FILE)

## Impute missing data and rename columns to SSScoring conventions

- Some DZ names are missing.  The city/town `name` will be imputed to the `DZ` column in those cases.
- Future revisions may use ISO-2 country codes instead of ISO-3.

In [ ]:
rawData.DZ = rawData.DZ.fillna(rawData.name)
data = pd.DataFrame(data = {
    'dropZone': rawData.DZ,
    'lat': rawData.lat,
    'lon': rawData.lon,
    'elevation': rawData.elevation,
    'location': rawData.name,
    'country': rawData.country,
})
data.sort_values(by='dropZone', ascending=True, inplace=True)
data.head(8)

## Preserve the normalized data

In [ ]:
data.to_csv('./ssscoring/resources/drop-zones-loc-elev.csv', sep=',')
!ls -l ./resources/drop-zones*

---
## Generate the synthetic Python module with the DZ data

This is necessary because Streamlit.io can't handle package resources natively for some reason or in a clean, Pythonic way.

In [ ]:
dropZones = pd.read_csv('./ssscoring/resources/drop-zones-loc-elev.csv', sep=',')

In [ ]:
dropZones.reset_index(drop=True)
dropZonesList = dropZones.to_dict('records')
for record in dropZonesList:
    del record['Unnamed: 0']

In [ ]:
with open('./ssscoring/dzdir.py', 'w') as outputStream:
    outputStream.write('"""\nAUTO-GENERATED - do not modify by hand.\n"""\n\n')
    outputStream.write('DROP_ZONES_LIST = %s' % dropZonesList)
    outputStream.write('\n"""\n@private\n"""\n')
    outputStream.write('\n\n# end of auto-generated file')

### Confirm the dzdir module works

In [ ]:
from ssscoring.dzdir import DROP_ZONES_LIST

In [ ]:
x = pd.DataFrame(DROP_ZONES_LIST)